In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
pd.set_option("display.max_column",100)
pd.set_option("expand_frame_repr",False)

In [3]:
off_train_file=r"D:\dataset\o2o\ccf_offline_stage1_train.csv"
off_test_file=r"D:\dataset\o2o\ccf_offline_stage1_test_revised.csv"
on_train_file=r"D:\dataset\o2o\ccf_online_stage1_train.csv"
off_train=pd.read_csv(off_train_file)
off_test=pd.read_csv(off_test_file)
on_train=pd.read_csv(on_train_file)

In [4]:
#工具类函数
def add_agg_feature_names(df,df_group,group_col,value_col,agg_ops,col_names):
     df_group[value_col]=df_group[value_col].astype("float")
     df_agg=pd.DataFrame(df_group.groupby(group_cols)[value_col].agg(agg_ops)).reset_index()
     df_agg.columns=group_col+col_names
     df=df.merge(df_agg,on=group_cols,how="left")
     return df

In [6]:
#统计特征处理函数
def add_agg_featurn(df,df_group,group_col,value_col,agg_ops,keyword):
    col_names=[]
    for op in agg_ops:
        col_names.append(keyword+"_"+value_col+"_"+op)
    df=add_agg_feature_names(df,df_group,group_cols,value_col,agg_ops,col_names)
    return df

In [8]:
#定义count函数
def add_count_new_feature(df,df_group,group_cols,new_feature_name):
    df_group[new_feature_name]=1
    df_group=df_group.group(group_cols).agg("sum").reset_index()
    df=df.merge(df_group,on=groupc_cols,how='left')
    return df

In [9]:
#商户相关特征群生成
def get_merchant_feature(feature):
    merchant=feature[['Merchant_id','Coupon_id','Distance','Date_received','Date']].copy() #[[]]表示生成的是一个DataFrame
    t=merchant[["Merchant_id"]].copy()
    #删除重复行数据
    t.drop_duplicates(inplace=True)
    #卖出的商品
    t1=merchant[merchant.Date!="null"][["Merchant_id"]].copy()
    merchant_feature=add_count_new_feature(t,t1,"Merchant_id","total_sales")
    #在每个商户销售中，使用优惠券的交易次数
    t2=merchant[(merchant.Data!="null")&(merchant.Coupon_id!="null")][["Merchant_id"]].copy()
    merchant_feature=add_count_new_feature(merchant_feature,t2,"Merchant_id","sale_use_coupon")
    #每个商户发放的优惠券总数
    t3=merchant[merchant.Coupon_id!="null"][["Merchant_id"]].copy()
    merchant_feature=add_count_new_feature(merchant_feature,t3,"Merchant_id","total_coupon")
    #在每个线下商户含有优惠券的交易中，统计和用户距离的最大值，最小值，平均值，中位数
    t4=merchant[(merchant.Data!="null")&(merchant.Coupon_id!="null")][["Merchant_id","Distance"]].copy()
    t4.Distance=t4.Distance.astype("int")
    merchant_feature=add_agg_feature(merchant_feature,t4,["Merchant_id"],"Distance",["min","max","mean","median"],"merchant")
    #将数据中的nan用0来替换
    user_merchant.user_merchant_buy_use_coupon=user_merchant.user_merchant_buy_use_coupon.replace(np.nan,0)
    user_merchant.user_merchant_buy_use_common=user_merchant.user_merchant_buy_common.replace(np.nan,0)
    return user_merchant


In [13]:
def f2(dataset,feature):
    result=add_discount(dataset)
    merchant_feature=get_merchant_feature(feature)
    result=result.merge(merchant_feature,on="Merchant_id",how="left")
    return result